In [ ]:
library(opploansanalytics)
load.packages()

## Spot Check Rules

In [ ]:
a = queryReportingLimit(
"
select
    lde.lead_id
    , lde.raw_lead ->> 'origination-id' as origination_id
    , lde.raw_lead
--  , usages.*
from
    lde4.leads as lde
--  left join
--      visine.report_usages as usages
--      on lde.lead_id = usages.origination_id
--      on lde.raw_lead ->> 'origination-id' = usages.origination_id
where
    lde.partnerid = 349
    and lde.apiversion = '5'
    and lde.lead_time >= '2020-05-20'::date
"
)

Clarity Report is no longer populated in LDE5. Need to figure out how to get associated Leads01 pull.

In [ ]:
getSumsAdmethodExclusion = function (timestart = '2019-10-01', timeend = Sys.Date(), admethod = 'LeadGroup') {

    queryReporting(paste0(
    "
    select
        lde.lead_time::date as lead_time
        , c_adm.name as admethod
        , count(*) as offered
        , sum(case when accepted then 1 else 0 end) as accepted
        , sum(case when c_app.id notnull then 1 else 0 end) as apps
        , sum(case when left(p_ap.denygrp,1) > '2' then 1 else 0 end) as qualified
        , sum(case when left(p_ap.denygrp,1) = '9' then 1 else 0 end) as funded
    from
        lde4.leads as lde
        inner join
            cloudlending.advertising_method as c_adm
            on lde.partnerid = c_adm.external_id
            and c_adm.name = '", admethod, "'
        left join
            cloudlending.applications as c_app
            on lde.lead_id = c_app.lde4_lead_id
            and c_app.type_formula = 'New'
        left join
            public.all_allapps as p_ap
            on c_app.name = p_ap.loanid
    where
        lde.lead_time >= '", timestart, "'::date
        and lde.lead_time < '", timeend, "'::date
        and lde.clarity_report -> 'xml_response' -> 'clear_credit_risk' ->> 'score' isnull
        and lde.clarity_report -> 'xml_response' -> 'clear_credit_risk' ->> 'number_of_bank_accounts' isnull
    group by
        1, 2
    order by
        2, 1 desc
    "
    ))
    
}

## Graph End Metrics

In [ ]:
getSums = function (timestart = '2019-10-01', timeend = '2020-06-08') {

    queryReporting(paste0(
    "
    select
        lde.lead_time::date as lead_time
        , c_adm.name as admethod
        , count(*) as offered
        , sum(case when accepted then 1 else 0 end) as accepted
        , sum(case when c_app.id notnull then 1 else 0 end) as apps
        , sum(case when left(p_ap.denygrp,1) > '2' then 1 else 0 end) as qualified
        , sum(case when left(p_ap.denygrp,1) = '9' then 1 else 0 end) as funded
    from
        lde4.leads as lde
        inner join
            cloudlending.advertising_method as c_adm
            on lde.partnerid = c_adm.external_id
--            and c_adm.name in ('LenderEdge 4', 'Even Financial 4', 'Monevo', 'Quin Street 4', 'LeadGroup')
            and c_adm.name in ('LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'LeadGroup', 'LeapTheory 4', 'Monevo', 'Avant', 'PersonalLoans.com 4', 'CreditKarma4', 'Lending Tree 4')
        left join
            cloudlending.applications as c_app
            on lde.lead_id = c_app.lde4_lead_id
            and c_app.type_formula = 'New'
        left join
            public.all_allapps as p_ap
            on c_app.name = p_ap.loanid
    where
        lde.lead_time >= '", timestart, "'::date
        and lde.lead_time < '", timeend, "'::date
    group by
        1, 2
    order by
        2, 1 desc
    "
    ))
    
}

In [ ]:
agg.sums = getSums()

In [ ]:
agg.sums %>% str()

In [ ]:
getAcceptToFund = function (agg.sums) {

    agg.sums %>% 
        filter(
            lead_time >= '2020-04-01' &
            lead_time < Sys.Date() - 5
        ) %>% 
        mutate(
            admethod =
                case_when(
                    admethod %in% c('LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup') ~ admethod,
                    TRUE ~ 'LDE Else'
                ) %>% 
                fct_relevel(
                    'LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup', 'LDE Else'
                ),
            
            accept.rate = accepted/offered,
            apply.rate = apps/accepted,
            qualified.rate = qualified/apps,
            funding.rate = funded/qualified,
            
            app.to.fund = funded/apps,
            accept.to.fund = funded/accepted
        ) %>% 
        filter(
            admethod != 'LDE Else'
        ) %>% 
        ggplot(
            mapping = aes(
                x = lead_time,
                y = accept.to.fund,
                color = admethod,
                fill = admethod
            )
        ) +
        geom_line() +
        geom_point() +
        geom_vline(
            xintercept = '2020-05-15' %>% as.Date() %>% as.numeric(),
            linetype = 4,
            color = 'black'
        ) +
        geom_smooth(
            method = 'loess'
        ) +
        facet_grid(
            cols = vars(admethod)
        ) +
        coord_fixed(
            ratio = 3000
        ) +
        scale_x_date(
            breaks = '2 weeks'
        ) +
        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            y = "Accept to Fund",
            x = "Date"
        ) +
        theme_bw() +
        theme(
            legend.position = 'none',
            axis.text.x = element_text(
                angle = -60,
                hjust = 0
            )
        ) 
    
}
# agg.sums %>% getAcceptToFund()

In [ ]:
getAppToFund = function (agg.sums) {

    agg.sums %>% 
        filter(
            lead_time >= '2020-04-01' &
            lead_time < Sys.Date() - 5 &
            admethod != 'CreditKarma4'
        ) %>% 
        mutate(
            admethod =
                case_when(
                    admethod %in% c('LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup') ~ admethod,
                    TRUE ~ 'LDE Else'
                ) %>% 
                fct_relevel(
                    'LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup', 'LDE Else'
                ),
            
            accept.rate = accepted/offered,
            apply.rate = apps/accepted,
            qualified.rate = qualified/apps,
            funding.rate = funded/qualified,
            
            app.to.fund = funded/apps,
            accept.to.fund = funded/accepted
        ) %>% 
        filter(
            admethod == 'LDE Else',
            app.to.fund < 0.5
        ) %>% 
        ggplot(
            mapping = aes(
                x = lead_time,
                y = app.to.fund,
                color = admethod,
                fill = admethod
            )
        ) +
        geom_line() +
        geom_point() +
        geom_vline(
            xintercept = '2020-05-15' %>% as.Date() %>% as.numeric(),
            linetype = 4,
            color = 'black'
        ) +
        geom_smooth(
            method = 'loess'
        ) +
        facet_grid(
            cols = vars(admethod)
        ) +
        coord_fixed(
            ratio = 600
        ) +
        scale_x_date(
            breaks = '2 weeks'
        ) +
        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            y = "App to Fund",
            x = "Date"
        ) +
        theme_bw() +
        theme(
            legend.position = 'none',
            axis.text.x = element_text(
                angle = -60,
                hjust = 0
            )
        ) 
    
}
# agg.sums %>% getAppToFund()

In [ ]:
getApplyRate = function (agg.sums) {

    agg.sums %>% 
        filter(
            lead_time >= '2020-04-01' &
            lead_time < Sys.Date() - 5
        ) %>% 
        mutate(
            admethod =
                case_when(
                    admethod %in% c('LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup') ~ admethod,
                    TRUE ~ 'LDE Else'
                ) %>% 
                fct_relevel(
                    'LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup', 'LDE Else'
                ),
            
            accept.rate = accepted/offered,
            apply.rate = apps/accepted,
            qualified.rate = qualified/apps,
            funding.rate = funded/qualified,
            
            app.to.fund = funded/apps,
            accept.to.fund = funded/accepted
        ) %>% 
        filter(
            admethod != 'LDE Else'
        ) %>% 
        ggplot(
            mapping = aes(
                x = lead_time,
                y = apply.rate,
                color = admethod,
                fill = admethod
            )
        ) +
        geom_line() +
        geom_point() +
        geom_vline(
            xintercept = '2020-05-15' %>% as.Date() %>% as.numeric(),
            linetype = 4,
            color = 'black'
        ) +
        geom_smooth(
            method = 'loess'
        ) +
        facet_grid(
            cols = vars(admethod)
        ) +
        coord_fixed(
            ratio = 800
        ) +
        scale_x_date(
            breaks = '2 weeks'
        ) +
        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            y = "Apply Rate",
            x = "Date"
        ) +
        theme_bw() +
        theme(
            legend.position = 'none',
            axis.text.x = element_text(
                angle = -60,
                hjust = 0
            )
        ) 
    
}
# agg.sums %>% getApplyRate()

In [ ]:
getQR = function (agg.sums) {

    agg.sums %>% 
        filter(
            lead_time >= '2020-04-01' &
            lead_time < Sys.Date() - 5
        ) %>% 
        mutate(
            admethod =
                case_when(
                    admethod %in% c('LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup') ~ admethod,
                    TRUE ~ 'LDE Else'
                ) %>% 
                fct_relevel(
                    'LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup', 'LDE Else'
                ),
            
            accept.rate = accepted/offered,
            apply.rate = apps/accepted,
            qualified.rate = qualified/apps,
            funding.rate = funded/qualified,
            
            app.to.fund = funded/apps,
            accept.to.fund = funded/accepted
        ) %>% 
        filter(
            admethod != 'LDE Else'
        ) %>% 
        ggplot(
            mapping = aes(
                x = lead_time,
                y = qualified.rate,
                color = admethod,
                fill = admethod
            )
        ) +
        geom_line() +
        geom_point() +
        geom_vline(
            xintercept = '2020-05-15' %>% as.Date() %>% as.numeric(),
            linetype = 4,
            color = 'black'
        ) +
        geom_smooth(
            method = 'loess'
        ) +
        facet_grid(
            cols = vars(admethod)
        ) +
        coord_fixed(
            ratio = 300
        ) +
        scale_x_date(
            breaks = '2 weeks'
        ) +
        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            y = "QR",
            x = "Date"
        ) +
        theme_bw() +
        theme(
            legend.position = 'none',
            axis.text.x = element_text(
                angle = -60,
                hjust = 0
            )
        ) 
    
}
# agg.sums %>% getQR()

In [ ]:
getFR = function (agg.sums) {

    agg.sums %>% 
        filter(
            lead_time >= '2020-04-01' &
            lead_time < Sys.Date() - 5
        ) %>% 
        mutate(
            admethod =
                case_when(
                    admethod %in% c('LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup') ~ admethod,
                    TRUE ~ 'LDE Else'
                ) %>% 
                fct_relevel(
                    'LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup', 'LDE Else'
                ),
            
            accept.rate = accepted/offered,
            apply.rate = apps/accepted,
            qualified.rate = qualified/apps,
            funding.rate = funded/qualified,
            
            app.to.fund = funded/apps,
            accept.to.fund = funded/accepted
        ) %>% 
        filter(
            admethod != 'LDE Else'
        ) %>% 
        ggplot(
            mapping = aes(
                x = lead_time,
                y = funding.rate,
                color = admethod,
                fill = admethod
            )
        ) +
        geom_line() +
        geom_point() +
        geom_vline(
            xintercept = '2020-05-15' %>% as.Date() %>% as.numeric(),
            linetype = 4,
            color = 'black'
        ) +
        geom_smooth(
            method = 'loess'
        ) +
        facet_grid(
            cols = vars(admethod)
        ) +
        coord_fixed(
            ratio = 300
        ) +
        scale_x_date(
            breaks = '2 weeks'
        ) +
        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            y = "FR",
            x = "Date"
        ) +
        theme_bw() +
        theme(
            legend.position = 'none',
            axis.text.x = element_text(
                angle = -60,
                hjust = 0
            )
        ) 
    
}
# agg.sums %>% getFR()

In [ ]:
getAcceptRate = function (agg.sums) {

    agg.sums %>% 
        filter(
            lead_time >= '2020-04-01' &
            lead_time < Sys.Date() - 5
        ) %>% 
        mutate(
            admethod =
                case_when(
                    admethod %in% c('LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup') ~ admethod,
                    TRUE ~ 'LDE Else'
                ) %>% 
                fct_relevel(
                    'LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup', 'LDE Else'
                ),
            
            accept.rate = accepted/offered,
            apply.rate = apps/accepted,
            qualified.rate = qualified/apps,
            funding.rate = funded/qualified,
            
            app.to.fund = funded/apps,
            accept.to.fund = funded/accepted
        ) %>% 
        filter(
            admethod != 'LDE Else'
        ) %>% 
        ggplot(
            mapping = aes(
                x = lead_time,
                y = accept.rate,
                color = admethod,
                fill = admethod
            )
        ) +
        geom_line() +
        geom_point() +
        geom_vline(
            xintercept = '2020-05-15' %>% as.Date() %>% as.numeric(),
            linetype = 4,
            color = 'black'
        ) +
        geom_smooth(
            method = 'loess'
        ) +
        facet_grid(
            cols = vars(admethod)
        ) +
        coord_fixed(
            ratio = 500
        ) +
        scale_x_date(
            breaks = '2 weeks'
        ) +
        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            y = "Accept Rate",
            x = "Date"
        ) +
        theme_bw() +
        theme(
            legend.position = 'none',
            axis.text.x = element_text(
                angle = -60,
                hjust = 0
            )
        ) 
    
}
# agg.sums %>% getAcceptRate()

In [ ]:
getFundedLoans = function (agg.sums) {

    agg.sums %>% 
        filter(
            lead_time >= '2020-04-01' &
            lead_time < Sys.Date() - 5
        ) %>% 
        mutate(
            admethod =
                case_when(
                    admethod %in% c('LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup') ~ admethod,
                    TRUE ~ 'LDE Else'
                ) %>% 
                fct_relevel(
                    'LenderEdge 4', 'Even Financial 4', 'Quin Street 4', 'Monevo', 'LeadGroup', 'LDE Else'
                ),
            
            accept.rate = accepted/offered,
            apply.rate = apps/accepted,
            qualified.rate = qualified/apps,
            funding.rate = funded/qualified,
            
            app.to.fund = funded/apps,
            accept.to.fund = funded/accepted
        ) %>% 
        filter(
            admethod != 'LDE Else'
        ) %>% 
        ggplot(
            mapping = aes(
                x = lead_time,
                y = funded %>% as.numeric,
                color = admethod,
                fill = admethod
            )
        ) +
        geom_line() +
        geom_point() +
        geom_vline(
            xintercept = '2020-05-15' %>% as.Date() %>% as.numeric(),
            linetype = 4,
            color = 'black'
        ) +
        geom_smooth(
            method = 'loess'
        ) +
        facet_grid(
            cols = vars(admethod)
        ) +
        coord_fixed(
            ratio = 10
        ) +
        scale_x_date(
            breaks = '2 weeks'
        ) +
        scale_y_continuous(
            labels = scales::number_format(accuracy = 1)
        ) +
        labs(
            y = "Funded Loans",
            x = "Date"
        ) +
        theme_bw() +
        theme(
            legend.position = 'none',
            axis.text.x = element_text(
                angle = -60,
                hjust = 0
            )
        ) 
    
}
agg.sums %>% getFundedLoans()